# 为用户推荐好友

In [12]:
# 导入包
import random
import math
import time
from tqdm import tqdm

## 一. 通用函数定义

In [13]:
# 定义装饰器，监控运行时间
def timmer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        res = func(*args, **kwargs)
        stop_time = time.time()
        print('Func %s, run time: %s' % (func.__name__, stop_time - start_time))
        return res
    return wrapper

### 1. 数据处理相关
1. load data
2. split data

在两个数据集上进行实验，一个是Slashdot，一个是Epinions，数据格式比较简单：每行两个节点(from-to)

In [14]:
class Dataset():
    
    def __init__(self, fp, sample=100000):
        # fp: data file path
        # sample: 只取部分数据集，-1则为全部
        self.data = self.loadData(fp, sample)
    
    def loadData(self, fp, sample):
        # 只取一个小数据集进行处理
        data = [f.strip().split('\t') for f in open(fp).readlines()[4:]]#前几行是数据集的描述
        if sample == -1: 
            return data
        else:
            random.shuffle(data)
            return data[:sample]
    
    def splitData(self, M, k, seed=1):
        '''
        :params: data, 加载的所有(user, item)数据条目
        :params: M, 划分的数目，最后需要取M折的平均
        :params: k, 本次是第几次划分，k~[0, M)
        :params: seed, random的种子数，对于不同的k应设置成一样的
        :return: train, test
        '''
        train, test = [], []
        random.seed(seed)
        for u, v in self.data:
            # 这里与书中的不一致，本人认为取M-1较为合理，因randint是左右都覆盖的
            if random.randint(0, M-1) == k:  
                test.append((u, v))
            else:
                train.append((u, v))

        # 处理成字典的形式，user->set(items)
        def convert_dict(data):
            data_dict = {} # 当前用户指向的用户
            data_dict_t = {} # 指向当前用户的用户
            for u, v in data:
                if u not in data_dict:
                    data_dict[u] = set()
                data_dict[u].add(v)
                if v not in data_dict_t:
                    data_dict_t[v] = set()
                data_dict_t[v].add(u)
            data_dict = {k: list(data_dict[k]) for k in data_dict}
            data_dict_t = {k: list(data_dict_t[k]) for k in data_dict_t}
            return data_dict, data_dict_t #注意出、入都要返回

        return convert_dict(train), convert_dict(test)[0] #训练集要包含用户出、入关系，而测试集只需要出的部分(给测试集的用户推荐好友)

### 2. 评价指标
1. Precision
2. Recall
3. Coverage
4. Popularity(Novelty)

In [15]:
class Metric():
    
    def __init__(self, train, test, GetRecommendation):
        '''
        :params: train, 训练数据
        :params: test, 测试数据
        :params: GetRecommendation, 为某个用户获取推荐物品的接口函数
        '''
        self.train = train
        self.test = test
        self.GetRecommendation = GetRecommendation
        self.recs = self.getRec()
        
    # 为test中的每个用户进行推荐
    def getRec(self):
        recs = {}
        for user in self.test:
            rank = self.GetRecommendation(user)
            recs[user] = rank
        return recs
        
    # 定义精确率指标计算方式
    def precision(self):
        all, hit = 0, 0
        for user in self.test:
            test_users = set(self.test[user])
            rank = self.recs[user]
            for v, score in rank:
                if v in test_users:
                    hit += 1
            all += len(rank)
        return round(hit / all * 100, 2) if all > 0 else 0
    
    # 定义召回率指标计算方式
    def recall(self):
        all, hit = 0, 0
        for user in self.test:
            test_users = set(self.test[user])
            rank = self.recs[user]
            for v, score in rank:
                if v in test_users:
                    hit += 1
            all += len(test_users)
        return round(hit / all * 100, 2) if all > 0 else 0
    
    def eval(self):
        metric = {'Precision': self.precision(),
                  'Recall': self.recall()}
        print('Metric:', metric)
        return metric

## 二. 算法实现
主要是不同的用户相似度计算方法
1. OUT
2. IN
3. OUT_IN
4. OUT_IN_Cosine

In [16]:
# 1. 利用用户出度计算相似度
def OUT(train, N):
    '''
    :params: train, 训练数据集（包含出和入的）
    :params: N, 超参数，设置取TopN推荐用户数目
    :return: GetRecommendation，推荐接口函数
    '''
    
    G, GT = train #出、入两部分(出：用户关注了哪些人； 入：用户有哪些粉丝)
    
    def GetRecommendation(user):
        if user not in G: return []
        # 根据相似度推荐N个未见过的
        user_sim = {}
        user_friends = set(G[user])
        for u in G[user]:
            if u not in GT: continue
            for v in GT[u]: #user与v共同关注了u
                if v != user and v not in user_friends:
                    if v not in user_sim:
                        user_sim[v] = 0
                    user_sim[v] += 1 #累计user与v共同关注的人数
        user_sim = {v: user_sim[v] / math.sqrt(len(G[user] * len(G[v]))) for v in user_sim} #求共同关注比例
        return list(sorted(user_sim.items(), key=lambda x: x[1], reverse=True))[:N]
    
    return GetRecommendation

In [17]:
# 2. 利用用户入度计算相似度
def IN(train, N):
    '''
    :params: train, 训练数据集（包含出和入的）
    :params: N, 超参数，设置取TopN推荐用户数目
    :return: GetRecommendation，推荐接口函数
    '''
    
    G, GT = train
        
    def GetRecommendation(user):
        if user not in GT: return []
        # 根据相似度推荐N个未见过的
        user_sim = {}
        user_friends = set(G[user]) if user in G else set()
        for u in GT[user]:
            if u not in G: continue
            for v in G[u]: #u是user和v的共同粉丝
                if v != user and v not in user_friends:
                    if v not in user_sim:
                        user_sim[v] = 0
                    user_sim[v] += 1 #累计user和v的共同粉丝数
        user_sim = {v: user_sim[v] / math.sqrt(len(GT[user] * len(GT[v]))) for v in user_sim} #求共同粉丝比例
        return list(sorted(user_sim.items(), key=lambda x: x[1], reverse=True))[:N]
    
    return GetRecommendation

In [18]:
# 3. 利用用户出度和入度进行计算，但没有考虑到热门入度(粉丝多的)用户的惩罚
def OUT_IN(train, N):
    '''
    :params: train, 训练数据集（包含出和入的）
    :params: N, 超参数，设置取TopN推荐用户数目
    :return: GetRecommendation，推荐接口函数
    '''
    
    G, GT = train
    
    def GetRecommendation(user):
        if user not in G: return []
        # 根据相似度推荐N个未见过的
        user_sim = {}
        user_friends = set(G[user])
        #求用户user关注的用户u中有多大比例是v的粉丝
        for u in G[user]:
            if u not in G: continue
            for v in G[u]:
                if v != user and v not in user_friends:
                    if v not in user_sim:
                        user_sim[v] = 0
                    user_sim[v] += 1
        user_sim = {v: user_sim[v] / len(G[user]) for v in user_sim}
        return list(sorted(user_sim.items(), key=lambda x: x[1], reverse=True))[:N]
    
    return GetRecommendation

In [19]:
# 4. 利用用户出度和入度的余弦相似度进行计算
def OUT_IN_Cosine(train, N):
    '''
    :params: train, 训练数据集（包含出和入的）
    :params: N, 超参数，设置取TopN推荐用户数目
    :return: GetRecommendation，推荐接口函数
    '''
    
    G, GT = train
    
    def GetRecommendation(user):
        if user not in G: return []
        # 根据相似度推荐N个未见过的
        user_sim = {}
        user_friends = set(G[user])
        for u in G[user]:
            if u not in G: continue
            for v in G[u]:
                if v != user and v not in user_friends:
                    if v not in user_sim:
                        user_sim[v] = 0
                    user_sim[v] += 1
        user_sim = {v: user_sim[v] / math.sqrt(len(G[user]) * len(GT[v])) for v in user_sim}
        return list(sorted(user_sim.items(), key=lambda x: x[1], reverse=True))[:N]
    
    return GetRecommendation

## 三. 实验

在两个数据集上进行实验，一个是Slashdot，一个是Epinions
1. OUT实验
2. IN实验
3. OUT_IN实验
4. OUT_IN_Cosine实验

M=10, N=10

In [20]:
class Experiment():
    def __init__(self,
                 M,
                 N,
                 fp='../dataset/slashdot/soc-Slashdot0902.txt',
                 rt='OUT'):
        '''
        :params: M, 进行多少次实验
        :params: N, TopN推荐用户的个数
        :params: fp, 数据文件路径
        :params: rt, 推荐算法类型
        '''
        self.M = M
        self.N = N
        self.fp = fp
        self.rt = rt
        self.alg = {'OUT': OUT, 'IN': IN, \
                    'OUT_IN': OUT_IN, 'OUT_IN_Cosine': OUT_IN_Cosine}

    # 定义单次实验
    def worker(self, train, test):
        '''
        :params: train, 训练数据集
        :params: test, 测试数据集
        :return: 各指标的值
        '''
        getRecommendation = self.alg[self.rt](train, self.N)
        metric = Metric(train[0], test, getRecommendation)
        return metric.eval()

    # 多次实验取平均
    def run(self):
        metrics = {'Precision': 0, 'Recall': 0}
        dataset = Dataset(self.fp, -1)
        for ii in range(self.M):
            train, test = dataset.splitData(self.M, ii)
            print('Experiment {}:'.format(ii))
            metric = self.worker(train, test)
            metrics = {k: metrics[k] + metric[k] for k in metrics}
        metrics = {k: metrics[k] / self.M for k in metrics}
        print('Average Result (M={}, N={}, alg={}): {}'.format(\
                              self.M, self.N, self.rt, metrics))

In [21]:
# 1. Slashdot数据集实验
M, N = 10, 10
for alg in ['OUT', 'IN', 'OUT_IN', 'OUT_IN_Cosine']:
    exp = Experiment(M, N, fp='../dataset/slashdot/soc-Slashdot0902.txt', rt=alg)
    exp.run()

Experiment 0:
Metric: {'Precision': 2.13, 'Recall': 6.21}
Experiment 1:
Metric: {'Precision': 2.08, 'Recall': 6.06}
Experiment 2:
Metric: {'Precision': 2.11, 'Recall': 6.12}
Experiment 3:
Metric: {'Precision': 2.07, 'Recall': 6.03}
Experiment 4:
Metric: {'Precision': 2.12, 'Recall': 6.16}
Experiment 5:
Metric: {'Precision': 2.07, 'Recall': 6.05}
Experiment 6:
Metric: {'Precision': 2.06, 'Recall': 5.98}
Experiment 7:
Metric: {'Precision': 2.13, 'Recall': 6.2}
Experiment 8:
Metric: {'Precision': 2.08, 'Recall': 6.03}
Experiment 9:
Metric: {'Precision': 2.13, 'Recall': 6.19}
Average Result (M=10, N=10, alg=OUT): {'Precision': 2.098, 'Recall': 6.103}
Experiment 0:
Metric: {'Precision': 1.58, 'Recall': 5.03}
Experiment 1:
Metric: {'Precision': 1.56, 'Recall': 4.93}
Experiment 2:
Metric: {'Precision': 1.55, 'Recall': 4.89}
Experiment 3:
Metric: {'Precision': 1.55, 'Recall': 4.91}
Experiment 4:
Metric: {'Precision': 1.6, 'Recall': 5.05}
Experiment 5:
Metric: {'Precision': 1.57, 'Recall': 4.98

In [22]:
# 2. Epinions数据集实验
M, N = 10, 10
for alg in ['OUT', 'IN', 'OUT_IN', 'OUT_IN_Cosine']:
    exp = Experiment(M, N, fp='../dataset/epinions/soc-Epinions1.txt', rt=alg)
    exp.run()

Experiment 0:
Metric: {'Precision': 2.82, 'Recall': 7.74}
Experiment 1:
Metric: {'Precision': 2.75, 'Recall': 7.52}
Experiment 2:
Metric: {'Precision': 2.77, 'Recall': 7.59}
Experiment 3:
Metric: {'Precision': 2.73, 'Recall': 7.53}
Experiment 4:
Metric: {'Precision': 2.7, 'Recall': 7.45}
Experiment 5:
Metric: {'Precision': 2.74, 'Recall': 7.55}
Experiment 6:
Metric: {'Precision': 2.72, 'Recall': 7.46}
Experiment 7:
Metric: {'Precision': 2.79, 'Recall': 7.69}
Experiment 8:
Metric: {'Precision': 2.74, 'Recall': 7.49}
Experiment 9:
Metric: {'Precision': 2.7, 'Recall': 7.47}
Average Result (M=10, N=10, alg=OUT): {'Precision': 2.7459999999999996, 'Recall': 7.5489999999999995}
Experiment 0:
Metric: {'Precision': 2.94, 'Recall': 7.61}
Experiment 1:
Metric: {'Precision': 2.92, 'Recall': 7.54}
Experiment 2:
Metric: {'Precision': 3.0, 'Recall': 7.74}
Experiment 3:
Metric: {'Precision': 2.92, 'Recall': 7.61}
Experiment 4:
Metric: {'Precision': 2.91, 'Recall': 7.59}
Experiment 5:
Metric: {'Precisi

## 四. 实验结果

1. 部分(100000)数据集：

(1). Slashdot数据集实验

    Average Result (M=10, N=10, alg=OUT): {'Precision': 0.077, 'Recall': 0.344}
    
    Average Result (M=10, N=10, alg=IN): {'Precision': 0.06, 'Recall': 0.265}
    
    Average Result (M=10, N=10, alg=OUT_IN): {'Precision': 0.081, 'Recall': 0.35}
    
    Average Result (M=10, N=10, alg=OUT_IN_Cosine): {'Precision': 0.021, 'Recall': 0.095}
    
(2). Epinions数据集实验

    Average Result (M=10, N=10, alg=OUT): {'Precision': 0.175, 'Recall': 0.749}
    
    Average Result (M=10, N=10, alg=IN): {'Precision': 0.215, 'Recall': 0.827}
    
    Average Result (M=10, N=10, alg=OUT_IN): {'Precision': 0.54, 'Recall': 2.294}

    Average Result (M=10, N=10, alg=OUT_IN_Cosine): {'Precision': 0.223, 'Recall': 0.957}
    
2. 完整数据集：

(1). Slashdot数据集实验

    Average Result (M=10, N=10, alg=OUT): {'Precision': 2.098, 'Recall': 6.103}
    
    Average Result (M=10, N=10, alg=IN): {'Precision': 1.574, 'Recall': 4.976999999999999}
    
    Average Result (M=10, N=10, alg=OUT_IN): {'Precision': 1.052, 'Recall': 3.0119999999999996}
    
    Average Result (M=10, N=10, alg=OUT_IN_Cosine): {'Precision': 0.6280000000000001, 'Recall': 1.796}
    
(2). Epinions数据集实验

    Average Result (M=10, N=10, alg=OUT): {'Precision': 2.7459999999999996, 'Recall': 7.5489999999999995}
    
    Average Result (M=10, N=10, alg=IN): {'Precision': 2.936, 'Recall': 7.615}
    
    Average Result (M=10, N=10, alg=OUT_IN): {'Precision': 4.365, 'Recall': 11.928999999999998}

    Average Result (M=10, N=10, alg=OUT_IN_Cosine): {'Precision': 3.2040000000000006, 'Recall': 8.760000000000002}

## 五. 总结
1. 要注意会出现KeyError的情况，需常加判断，提升代码的鲁棒性
2. 在部分(100000)数据上的结果性能很差，尝试使用完整的数据集
3. 在完整数据集上的表现还可以，在slashdot数据集上Wout取得了最好的性能，而在Epinions数据集上Wout_in取得了最好的性能，这一点与书中是一致的，但整体性能比书中差很多（详见p180），具体原因尚未可知

## 附：运行日志（请双击看）

1. Slashdot数据集实验
Experiment 0:
Metric: {'Precision': 0.08, 'Recall': 0.36}
Experiment 1:
Metric: {'Precision': 0.07, 'Recall': 0.31}
Experiment 2:
Metric: {'Precision': 0.07, 'Recall': 0.29}
Experiment 3:
Metric: {'Precision': 0.09, 'Recall': 0.41}
Experiment 4:
Metric: {'Precision': 0.08, 'Recall': 0.36}
Experiment 5:
Metric: {'Precision': 0.09, 'Recall': 0.41}
Experiment 6:
Metric: {'Precision': 0.07, 'Recall': 0.3}
Experiment 7:
Metric: {'Precision': 0.07, 'Recall': 0.32}
Experiment 8:
Metric: {'Precision': 0.08, 'Recall': 0.37}
Experiment 9:
Metric: {'Precision': 0.07, 'Recall': 0.31}
Average Result (M=10, N=10, alg=OUT): {'Precision': 0.077, 'Recall': 0.344}
Experiment 0:
Metric: {'Precision': 0.05, 'Recall': 0.2}
Experiment 1:
Metric: {'Precision': 0.07, 'Recall': 0.3}
Experiment 2:
Metric: {'Precision': 0.08, 'Recall': 0.36}
Experiment 3:
Metric: {'Precision': 0.05, 'Recall': 0.23}
Experiment 4:
Metric: {'Precision': 0.06, 'Recall': 0.25}
Experiment 5:
Metric: {'Precision': 0.08, 'Recall': 0.36}
Experiment 6:
Metric: {'Precision': 0.04, 'Recall': 0.18}
Experiment 7:
Metric: {'Precision': 0.05, 'Recall': 0.21}
Experiment 8:
Metric: {'Precision': 0.07, 'Recall': 0.32}
Experiment 9:
Metric: {'Precision': 0.05, 'Recall': 0.24}
Average Result (M=10, N=10, alg=IN): {'Precision': 0.06000000000000001, 'Recall': 0.265}
Experiment 0:
Metric: {'Precision': 0.07, 'Recall': 0.31}
Experiment 1:
Metric: {'Precision': 0.08, 'Recall': 0.35}
Experiment 2:
Metric: {'Precision': 0.08, 'Recall': 0.34}
Experiment 3:
Metric: {'Precision': 0.1, 'Recall': 0.43}
Experiment 4:
Metric: {'Precision': 0.08, 'Recall': 0.35}
Experiment 5:
Metric: {'Precision': 0.11, 'Recall': 0.49}
Experiment 6:
Metric: {'Precision': 0.06, 'Recall': 0.27}
Experiment 7:
Metric: {'Precision': 0.07, 'Recall': 0.3}
Experiment 8:
Metric: {'Precision': 0.09, 'Recall': 0.37}
Experiment 9:
Metric: {'Precision': 0.07, 'Recall': 0.29}
Average Result (M=10, N=10, alg=OUT_IN): {'Precision': 0.081, 'Recall': 0.35}
Experiment 0:
Metric: {'Precision': 0.02, 'Recall': 0.1}
Experiment 1:
Metric: {'Precision': 0.03, 'Recall': 0.11}
Experiment 2:
Metric: {'Precision': 0.02, 'Recall': 0.09}
Experiment 3:
Metric: {'Precision': 0.02, 'Recall': 0.08}
Experiment 4:
Metric: {'Precision': 0.03, 'Recall': 0.15}
Experiment 5:
Metric: {'Precision': 0.02, 'Recall': 0.1}
Experiment 6:
Metric: {'Precision': 0.02, 'Recall': 0.08}
Experiment 7:
Metric: {'Precision': 0.01, 'Recall': 0.06}
Experiment 8:
Metric: {'Precision': 0.02, 'Recall': 0.1}
Experiment 9:
Metric: {'Precision': 0.02, 'Recall': 0.08}
Average Result (M=10, N=10, alg=OUT_IN_Cosine): {'Precision': 0.020999999999999998, 'Recall': 0.095}

2. Epinions数据集实验
Experiment 0:
Metric: {'Precision': 0.15, 'Recall': 0.63}
Experiment 1:
Metric: {'Precision': 0.21, 'Recall': 0.9}
Experiment 2:
Metric: {'Precision': 0.18, 'Recall': 0.79}
Experiment 3:
Metric: {'Precision': 0.16, 'Recall': 0.68}
Experiment 4:
Metric: {'Precision': 0.17, 'Recall': 0.73}
Experiment 5:
Metric: {'Precision': 0.19, 'Recall': 0.84}
Experiment 6:
Metric: {'Precision': 0.16, 'Recall': 0.67}
Experiment 7:
Metric: {'Precision': 0.17, 'Recall': 0.72}
Experiment 8:
Metric: {'Precision': 0.17, 'Recall': 0.73}
Experiment 9:
Metric: {'Precision': 0.19, 'Recall': 0.8}
Average Result (M=10, N=10, alg=OUT): {'Precision': 0.175, 'Recall': 0.7489999999999999}
Experiment 0:
Metric: {'Precision': 0.2, 'Recall': 0.77}
Experiment 1:
Metric: {'Precision': 0.2, 'Recall': 0.78}
Experiment 2:
Metric: {'Precision': 0.25, 'Recall': 0.96}
Experiment 3:
Metric: {'Precision': 0.22, 'Recall': 0.85}
Experiment 4:
Metric: {'Precision': 0.19, 'Recall': 0.74}
Experiment 5:
Metric: {'Precision': 0.22, 'Recall': 0.83}
Experiment 6:
Metric: {'Precision': 0.23, 'Recall': 0.89}
Experiment 7:
Metric: {'Precision': 0.22, 'Recall': 0.85}
Experiment 8:
Metric: {'Precision': 0.19, 'Recall': 0.72}
Experiment 9:
Metric: {'Precision': 0.23, 'Recall': 0.88}
Average Result (M=10, N=10, alg=IN): {'Precision': 0.215, 'Recall': 0.827}
Experiment 0:
Metric: {'Precision': 0.5, 'Recall': 2.13}
Experiment 1:
Metric: {'Precision': 0.53, 'Recall': 2.25}
Experiment 2:
Metric: {'Precision': 0.54, 'Recall': 2.26}
Experiment 3:
Metric: {'Precision': 0.52, 'Recall': 2.22}
Experiment 4:
Metric: {'Precision': 0.53, 'Recall': 2.24}
Experiment 5:
Metric: {'Precision': 0.59, 'Recall': 2.51}
Experiment 6:
Metric: {'Precision': 0.56, 'Recall': 2.37}
Experiment 7:
Metric: {'Precision': 0.55, 'Recall': 2.36}
Experiment 8:
Metric: {'Precision': 0.52, 'Recall': 2.21}
Experiment 9:
Metric: {'Precision': 0.56, 'Recall': 2.39}
Average Result (M=10, N=10, alg=OUT_IN): {'Precision': 0.54, 'Recall': 2.294}
Experiment 0:
Metric: {'Precision': 0.19, 'Recall': 0.82}
Experiment 1:
Metric: {'Precision': 0.21, 'Recall': 0.91}
Experiment 2:
Metric: {'Precision': 0.23, 'Recall': 0.96}
Experiment 3:
Metric: {'Precision': 0.26, 'Recall': 1.11}
Experiment 4:
Metric: {'Precision': 0.22, 'Recall': 0.94}
Experiment 5:
Metric: {'Precision': 0.26, 'Recall': 1.09}
Experiment 6:
Metric: {'Precision': 0.21, 'Recall': 0.9}
Experiment 7:
Metric: {'Precision': 0.24, 'Recall': 1.05}
Experiment 8:
Metric: {'Precision': 0.18, 'Recall': 0.79}
Experiment 9:
Metric: {'Precision': 0.23, 'Recall': 1.0}
Average Result (M=10, N=10, alg=OUT_IN_Cosine): {'Precision': 0.223, 'Recall': 0.9570000000000001}